# Arbitrage auto detection using matrices
 
 https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1096549

In [1]:
import urllib
import urllib.request
import json

import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [4]:
binance_uri = binance_url + binance_endpoints['exchange_info'][1]

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))

df_symbols = pd.DataFrame(JSON_object['symbols'])
df_symbols = df_symbols[df_symbols['status']=='TRADING']

In [5]:
binance_uri = binance_url + binance_endpoints['price'][1] #+ '?symbol=BTCEUR'

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))
df_prices = pd.DataFrame(JSON_object)

In [6]:
symbols = df_symbols['baseAsset'].unique()

In [7]:
df_prices['first_symbol'] = np.NaN
for i in range(df_prices['symbol'].apply(len).max()):
    mask = df_prices['symbol'].str[0:i].isin(symbols)
    df_prices.loc[mask, 'first_symbol'] = df_prices[mask]['symbol'].str[0:i]
    
df_prices['second_symbol'] = np.NaN
for i in range(df_prices['symbol'].apply(len).max()):
    mask = df_prices['symbol'].str[-(i+1):].isin(symbols)
    df_prices.loc[mask, 'second_symbol'] = df_prices[mask]['symbol'].str[-(i+1):]
    
df_prices = df_prices.drop(index=df_prices[df_prices['first_symbol'].isna()].index)
df_prices = df_prices.drop(index=df_prices[df_prices['second_symbol'].isna()].index)

In [31]:
change_matrix = pd.DataFrame(np.identity(len(symbols)), index=symbols, columns=symbols)

for index, row in df_prices.iterrows():
    change_matrix.loc[row['first_symbol'], row['second_symbol']] = row['price']
    
change_matrix = change_matrix.astype('float')

In [9]:
# Complete the matrix with the reciprocal
matrix = np.tril(change_matrix.to_numpy())
matrix_mask = matrix!=0
upper_matrix = np.reciprocal(matrix, where=matrix_mask).T

matrix = np.triu(change_matrix.to_numpy())
matrix_mask = matrix!=0
lower_matrix = np.reciprocal(matrix, where=matrix_mask).T

change_matrix = change_matrix + upper_matrix + lower_matrix - np.identity(len(change_matrix))*2

In [10]:
change_matrix

,ETH,LTC,BNB,NEO,QTUM,EOS,SNT,BNT,GAS,BTC,...,BADGER,FIS,OM,POND,DEGO,ALICE,BIFI,LINA,PERP,RAMP
ETH,1.000,8.892,0.000,37.877,206.148,396.356,13758.943,227.380,0.000,0.030,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
LTC,0.114,1.000,0.734,0.000,0.000,0.000,0.000,0.000,0.000,0.003,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
BNB,0.000,1.362,1.000,6.049,10.516,61.366,0.000,0.000,0.000,0.005,...,0.000,0.000,0.000,0.000,0.000,0.000,6.917,0.000,0.000,0.000
NEO,0.026,0.000,0.170,1.000,0.000,0.000,0.000,0.000,0.000,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
QTUM,0.005,0.000,0.096,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ALICE,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
BIFI,0.000,0.000,6.769,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
LINA,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000
PERP,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000


In [11]:
change_matrix.loc['BTC', 'EUR'] / change_matrix.loc['EUR', 'LTC'] * change_matrix.loc['LTC', 'BTC']

26473.4551113711

In [12]:
change_matrix.loc['BTC', 'EUR']

47489.29

In [13]:
change_matrix.loc['EUR', 'LTC']

0.006145714900285776

In [62]:
change_matrix.loc['LTC', 'BTC']

0.003426

In [15]:
47370.06 / 163.03913372752754

290.54410997524843

In [16]:
df_prices[df_prices['first_symbol']=='LTC']

,symbol,price,first_symbol,second_symbol
1,LTCBTC,0.00342600,LTC,BTC
189,LTCETH,0.11392000,LTC,ETH
190,LTCUSDT,192.15000000,LTC,USDT
191,LTCBNB,0.73400000,LTC,BNB
450,LTCTUSD,192.36000000,LTC,TUSD
451,LTCPAX,191.74000000,LTC,PAX
452,LTCUSDC,191.92000000,LTC,USDC
640,LTCBUSD,192.00000000,LTC,BUSD
1154,LTCEUR,162.71500000,LTC,EUR


---

In [38]:
symbols_lite = ['BTC','ETH','EUR','USDT']
matrix_lite = change_matrix.loc[symbols_lite, symbols_lite]
matrix_lite

,BTC,ETH,EUR,USDT
BTC,1.000,0.000,47489.290,56081.850
ETH,0.030,1.000,1427.670,12.200
EUR,0.000,0.000,1.000,1.181
USDT,0.000,0.000,0.000,1.000


In [39]:
matrix = np.tril(matrix_lite.to_numpy())
matrix_mask = matrix!=0
upper_matrix = np.reciprocal(matrix, where=matrix_mask).T

In [44]:
matrix = np.triu(matrix_lite.to_numpy())
matrix_mask = matrix!=0
lower_matrix = np.reciprocal(matrix, where=matrix_mask).T

In [56]:
matrix_lite = matrix_lite + pd.DataFrame(upper_matrix + lower_matrix - np.identity(len(matrix_lite))*2, index=symbols_lite, columns=symbols_lite)

In [57]:
matrix_lite

,BTC,ETH,EUR,USDT
BTC,1.000,33.280,47489.290,56081.850
ETH,0.030,1.000,1427.670,12.200
EUR,47489.290,1427.671,1.000,1.181
USDT,56081.850,12.282,2.028,1.000


In [58]:
matrix_lite.loc['BTC', 'ETH'] 

33.280282813798834

In [59]:
matrix_lite.loc['ETH', 'EUR'] 

1427.67

In [64]:
matrix_lite.loc['EUR', 'BTC'] 

47489.29002105738

In [63]:
np.linalg.eig(matrix_lite.to_numpy())

(array([ 73495.37129127, -73491.36632054,   1081.52492549,  -1081.52989623]),
 array([[-0.70704711, -0.7070527 , -0.00898256, -0.0089551 ],
        [-0.00896831, -0.00896795,  0.70724786,  0.70686894],
        [-0.45705005,  0.45704879,  0.53918387, -0.53988733],
        [-0.53954536,  0.53953912, -0.45716577,  0.45692207]]))